In [1]:
from datetime import date

from dapla_metadata.variable_definitions.generated.vardef_client.models.contact import (
    Contact,
)
from dapla_metadata.variable_definitions.generated.vardef_client.models.draft import (
    Draft,
)
from dapla_metadata.variable_definitions.generated.vardef_client.models.update_draft import (
    UpdateDraft,
)
from dapla_metadata.variable_definitions.vardef import Vardef

In [ ]:
# If necessary check token
from dapla_metadata.variable_definitions.config import get_oidc_token
from dapla_metadata.variable_definitions.config import get_vardef_host

print(get_oidc_token())
print(get_vardef_host())

In [3]:
### data for creating a draft variable definition
### Run this for validating your data

draft = Draft(
    name = {
        "nb": "",
        "nn": "",
        "en": "",
    },
    short_name= "fin",
    definition= {
        "nb": "",
        "nn": "",
        "en": "",
    },
    classification_reference="91",
    unit_types=[
        "01",
    ],
    subject_fields=[
        "al",
    ],
    contains_special_categories_of_personal_data=False,
    measurement_type=None,
    valid_from=date(2024,12,18),
    external_reference_uri=None,
    comment=None,
    related_variable_definition_uris=None,
    contact=None,
)

In [5]:
### Create draft variable definition and save in a variable
# Run this only once per session

my_draft = Vardef.create_draft(draft)

In [ ]:
# check details
print(my_draft.contact)

In [23]:
# Update draft data add contact
# validate data

my_contact = Contact(
    title={
        "nb": "res",
        "nn": "res",
        "en": "res",
        },
    email="sibby@ssb.no",
)
update_contact = UpdateDraft(
    contact=my_contact,
)

In [ ]:
my_draft.update_draft(update_draft=update_contact)

In [12]:
# update your variable
my_draft = Vardef.get_variable_definition(my_draft.id)

In [ ]:
# Test draft exceptions

In [ ]:
# Publish
from dapla_metadata.variable_definitions.generated.vardef_client.models.variable_status import VariableStatus


update_status = UpdateDraft(
    variable_status=VariableStatus.PUBLISHED_INTERNAL,
)

my_draft.update_draft(update_status)

In [29]:
# create patch
from dapla_metadata.variable_definitions.generated.vardef_client.models.patch import (
    Patch,
)

add_unit_types_patch = Patch(
    unit_types=["01", "02"],
)

my_draft.create_patch(add_unit_types_patch)
my_draft = Vardef.get_variable_definition(my_draft.id)

In [ ]:
# patch exceptions
invalid_subject_fields = Patch(
    subject_fields=["hahah"],
)

my_draft.create_patch(invalid_subject_fields)

In [ ]:
# read details
Vardef.get_variable_definition(my_draft.id).subject_fields

In [ ]:
# create validity period
from dapla_metadata.variable_definitions.generated.vardef_client.models.validity_period import (
    ValidityPeriod,
)

invalid_validity_period = ValidityPeriod(
    name={
        "nb": "new name",
    },
    valid_from=date(2025,4,5),
)

my_draft.create_validity_period(invalid_validity_period)

In [ ]:
# validity period exceptions
# correct validity period
# check languages

valid_validity_period = ValidityPeriod(
    definition={
        "nb": "ny definisjon",
        "nn": "ny definisjon",
        "en": "new definition",
    },
    valid_from=date(2025,4,5),
)
my_draft.create_validity_period(valid_validity_period)


In [43]:
print(Vardef.get_variable_definition(my_draft.id))

  Expected `LanguageStringType` but got `dict` with value `{'nb': '', 'nn': '', 'en': ''}` - serialized value may not be as expected
  Expected `LanguageStringType` but got `dict` with value `{'nb': 'ny definisjon', '... 'en': 'new definition'}` - serialized value may not be as expected
  Expected `Owner` but got `dict` with value `{'team': 'dapla-felles', ...pla-felles-developers']}` - serialized value may not be as expected
  Expected `Contact` but got `dict` with value `{'title': {'nb': 'res', '...'email': 'sibby@ssb.no'}` - serialized value may not be as expected, Category: UserWarning, Filename: /Users/cecilieseim/Library/Caches/pypoetry/virtualenvs/dapla-toolbelt-metadata-fJc960va-py3.12/lib/python3.12/site-packages/pydantic/main.py, Line: 477
{
  "id": "llg33thD",
  "patch_id": 8,
  "name": {
    "nb": "",
    "nn": "",
    "en": ""
  },
  "short_name": "fin",
  "definition": {
    "nb": "ny definisjon",
    "nn": "ny definisjon",
    "en": "new definition"
  },
  "classificatio

In [46]:
# check all patches
print(my_draft.list_patches())
print(my_draft.list_validity_periods())

[CompletePatchOutput(id='llg33thD', patch_id=1, name={'nb': '', 'nn': '', 'en': ''}, short_name='fin', definition={'nb': '', 'nn': '', 'en': ''}, classification_reference='91', unit_types=['01'], subject_fields=['al'], contains_special_categories_of_personal_data=False, variable_status=<VariableStatus.PUBLISHED_INTERNAL: 'PUBLISHED_INTERNAL'>, measurement_type=None, valid_from=datetime.date(2024, 12, 18), valid_until=None, external_reference_uri=None, comment=None, related_variable_definition_uris=None, owner={'team': 'dapla-felles', 'groups': ['dapla-felles-developers']}, contact={'title': {'nb': 'res', 'nn': 'res', 'en': 'res'}, 'email': 'sibby@ssb.no'}, created_at=datetime.datetime(2024, 12, 18, 13, 47, 50, 190000), created_by=None, last_updated_at=datetime.datetime(2024, 12, 18, 13, 51, 42, 802000), last_updated_by=None), CompletePatchOutput(id='llg33thD', patch_id=2, name={'nb': '', 'nn': '', 'en': ''}, short_name='fin', definition={'nb': '', 'nn': '', 'en': ''}, classification_re